In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd gdrive/MyDrive/24.7ai Project/Data

/content/gdrive/.shortcut-targets-by-id/1ufxCi8Sp-EWkSn5IcUhOV-zTDoVvzGZe/24.7ai Project/Data


In [ ]:
mappings = dict()

In [ ]:
import csv
import re

# We create a set of entities that we come across in the files 
# so that we know which ones we have data for
def get_seen_entities(lines):
  seen_entities = set()
  for line in lines:
    split_line = line.split('\t')
    head = split_line[0]
    tail = split_line[-1]
    seen_entities.add(head)
    seen_entities.add(tail)
  return seen_entities

# Using the entities that we've seen, we create a natural language
# version of the files if we have mappings for the entities within each line
def convert_to_nl(text, seen_entities, new_name):
  for raw_line in text:
    head = raw_line.split('[', 1)[1].split(']')[0]
    split_line = raw_line.split('\t')
    split_line[-1] = split_line[-1].split('|')
    tails = split_line[-1]
    if len(tails) == 1:
      tails[0] = tails[0].replace('\n', '')
    else:
      tails[-1] = tails[-1].replace('\n', '')

    count = 0

    valid_tails = []
    for i in tails:
      if i in mappings:
        i = i.replace(i, mappings[i])
        tails[count] = i
        valid_tails.append(i)
        count += 1
  
    valid_head = False
    if head in mappings:
      split_line[0] = split_line[0].replace(head, mappings[head])
      valid_head = True

    if len(valid_tails) > 0 and valid_head:
      if mappings[head] in seen_entities and not set(tails).isdisjoint(seen_entities):
        with open(new_name, 'a') as file:
          cleaned_tails = str(valid_tails).replace(", ","|").replace("[","").replace("]","").replace("'","")
          file.write(split_line[0] + '\t' + cleaned_tails + '\n')

In [ ]:
iter = 0

# We update our dictionary with the mapping between mid's 
# and their natural language names
with open("mid2name.tsv") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"):
      new_entity = line[0].lstrip('/').replace('/', '.')
      mappings[new_entity] = line[1]

with open('fb_natural_language_train.txt') as f:
    train_lines = f.readlines()

with open('fb_natural_language_test.txt') as f:
    test_lines = f.readlines()

with open('qa_train_webqsp.txt') as f:
    qa_train_lines = f.readlines()

with open('qa_test_webqsp.txt') as f:
    qa_test_lines = f.readlines()

In [ ]:
train_seen_entities = get_seen_entities(train_lines)
test_seen_entities = get_seen_entities(test_lines)

In [ ]:
convert_to_nl(qa_train_lines, train_seen_entities, "full_qa_train_nl.txt")

In [ ]:
convert_to_nl(qa_test_lines, test_seen_entities, "full_qa_test_nl.txt")